Amadéo David (44761700) - Marine Van Renterghem (31621700) - Sylvie Van Schendel (44841700)

### I.4 Résolution numérique du problème I.3 avec les données fournies

In [11]:
from scipy.optimize import linprog # import du solveur d'optimisation linéaire à utiliser
import matplotlib.pyplot as plt # pour graphiques éventuels
import numpy as np

In [6]:
def readFile(file_name):
    """
    Cette fonction lit un fichier formatté au type utilise dans les exemples du
    projet
    @pre                : Le fichier file_name est organise de la meme maniere
                          que les exemples donnes sur Moodle.
    
    @param file_name    : contient un String avec le nom du fichier a lire.
    
    @return n_points    : Contient un 'int' avec le nombre de points
    @return n_conduites : Contient un 'int' avec le nombre de conduites
    @return X           : Contient un 'numpy.array' avec les abscisses des points.
    @return Y           : Contient un 'numpy.array' avec les ordonnees des points.
    @return Z           : Contient un 'numpy.array' avec la hauteur des points.
    @return A           : Contient un 'numpy.array' avec la matrice d'incidence
                          des conduites.
    @return alpha       : Contient un 'float' avecla valeur de alpha.
    @return R           : Contient un 'float' avec le rayon des conduites.
    @return pa          : Contient un 'numpy.array' avec les points
                          d'approvisionnement.
    @return da_max      : Contient un 'numpy.array' avec les debits maximaux
                          extractibles.
    @return C           : Contient un 'numpy.array' avec les couts d'extraction.
    @return pc          : Contient un 'numpy.array' avec les points de
                          consommation.
    @return dc_min      : Contient un 'numpy.array' avec les debits minimaux
                          consommables.
    @return dc_max      : Contient un 'numpy.array' avec les debits maximaux
                          consommables.
    @return Pf          : Contient un 'float' avec le prix facture.
    @return pi          : Contient un 'numpy.array' avec les points
                          intermediaires.
    @return unit        : Contient un 'String' avec l'unite des coordonnes des
                          points.
    """
    with open(file_name,'r') as f:
        n_points    = (int)(f.readline().split()[0])
        n_conduites = (int)(f.readline().split()[0])
        f.readline()
        unit        = (f.readline().split(" = ")[1])
        
        coord       = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(n_points)))
        X           = coord[:,0]
        Y           = coord[:,1]
        Z           = coord[:,2]
        
        f.readline()
        f.readline()
        A           = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(n_points)))
        f.readline()
        
        UL_alpha    = f.readline().split(" = ")[1]
        UL_alpha    = UL_alpha.split()[0]
        UL2_alpha   = np.array(UL_alpha.split("^"))
        if(len(UL2_alpha) == 1):
            alpha   = (float)(UL2_alpha)
        else:
            alpha   = (float)(UL2_alpha[0]) ** (float)(UL2_alpha[1])
        
        R           = (float)(f.readline().split()[4])
        f.readline()
        
        UL_pa       = f.readline().split(" = [")[1]
        UL_pa       = UL_pa.split("]")[0]
        pa          = np.array(list(int(w) for w in UL_pa.split()[:]))
        
        UL_da_max   = f.readline().split(" = [")[1]
        UL_da_max   = UL_da_max.split("]")[0]
        da_max      = np.array(list(float(w) for w in UL_da_max.split()[:]))
        
        UL_C        = f.readline().split(" = [")[1]
        UL_C        = UL_C.split("]")[0]
        C           = np.array(list(float(w) for w in UL_C.split()[:]))
        
        f.readline()
        
        UL_pc       = f.readline().split(" = [")[1]
        UL_pc       = UL_pc.split("]")[0]
        pc          = np.array(list(int(w) for w in UL_pc.split()[:]))
        
        UL_dc_min   = f.readline().split(" = [")[1]
        UL_dc_min   = UL_dc_min.split("]")[0]
        dc_min      = np.array(list(float(w) for w in UL_dc_min.split()[:]))
        
        UL_dc_max   = f.readline().split(" = [")[1]
        UL_dc_max   = UL_dc_max.split("]")[0]
        dc_max      = np.array(list(float(w) for w in UL_dc_max.split()[:]))
        
        Pf          = (float)(f.readline().split()[3])
        
        f.readline()
        
        UL_pi       = f.readline().split("[")[1]
        UL_pi       = UL_pi.split("]")[0]
        pi          = np.array(list(int(w) for w in UL_pi.split()[:]))
        
    return (n_points, n_conduites, X, Y, Z, A, alpha, R, pa, da_max, C, pc, dc_min, dc_max, Pf, pi, unit)

In [46]:
#Fonction retournant [f,xC,xA] avec f la valeur de la fonction objectif maximale 
#xC un vecteur des valeurs optimales des débits aux points de consommation
#xA un vecteur des valeurs optimales des débits aux points d'approvisionnement
#Soit Pf le prix facturé
#Soit C le cout
#Soit DCmin et DCmax les valeurs minimales et maximales du débit aux points de consommation
#Soit DAmax les valeurs maximales du débit extractibles aux sources
def Analyse(nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi,alpha):
   
    h = abs(A.T@Z)      # le denivele de chaque conduite entre le pt de depart et d'arrivee.
    x=abs(A.T@X)        # la difference de x de chaque conduite entre le pt de depart et d'arrivee.
    y=abs(A.T@Y)        # la difference de y de chaque conduite entre le pt de depart et d'arrivee.
    L=np.sqrt(h*h+x*x+y*y)       
    print("This is h")
    print(h)
    print("This is x")
    print(x)
    print("This is y")
    print(y)
    print("This is L")
    print(L)
    print()
    
    A1     = np.concatenate((A[np.array(Pc)-1],-1* A[np.array(Pa)-1]))   # Tous les debits positifs
    A2     = A[np.array(Pc)-1]                                           # Uniquement les points de consommation.
    A3     = np.concatenate((A[np.array(Pc)-1], A[np.array(Pa)-1]))      # On laisse les debits aux points d'approvisionnement 
                                                                         # negatifs
    DebMax = alpha*R*R*h/L*A1   # Debit max sur les points de conso et approvisionnement
    DebMin = -1*alpha*R*R*h/L*A2 # Debit min (pts conso)
    print("This is DebMin")
    print(DebMin)
    print()
    print("This is DebMax")
    print(DebMax)
    print()
    
    #Sum    = np.sum(np.multiply(np.outer(np.ones(len(A1)),alpha*R*R*h/L),A3), axis=0)    
    #Aub    = np.concatenate((np.concatenate((DebMax,DebMin)),np.reshape(Sum, (1,np.shape(DebMax)[1]))))
    Aub = np.concatenate((DebMax,DebMin))
    Argent = np.concatenate((Pf,C))
    Eye=np.eye(len(Argent))*Argent
    print(Eye)
    c=-1*np.matmul(Argent,A3)*(alpha*R*R*h/L)    
    print(c)
    A5=A[np.array(Pi)-1] #On ne prend que les points 
    Aeq=A5*alpha*R*R*h/L #Points intemédiaires ont un débit resultant nul
    beq=np.zeros(len(Pi))
    #c=np.sum(alpha*R*R*h/L*A3*np.outer(np.concatenate((Pf,C)),np.ones(len(A3[0]))),axis=0)
    
    #b      = np.concatenate((np.concatenate((Dcmax,np.array(Damax))),np.concatenate((-1*np.array(Dcmin),[0]))))
    b      = np.concatenate((np.concatenate((Dcmax, Damax)), -1*Dcmin))
    
    bounds = np.concatenate((np.zeros((nConduites,1)),np.ones((nConduites,1))),axis=1) # limites (vecteur de (0,1) sur le
                                                                                       # nombre de conduites)
    print("This is c :")
    print(c)
    print()
    print("This is Aub")
    print(Aub)
    print()
    print("This is b")
    print(b)
    print()
    print("This is Aeq")
    print(Aeq)
    print()
    print("This is beq")
    print(beq)
    print()
    print("This is Bounds")
    print(bounds)
    print()
    res    = linprog(c, A_ub = Aub, b_ub = b, A_eq=Aeq, b_eq=beq,bounds=bounds,options={'tol': 1e-11})
    print(res)
    DebC=(A2*alpha*R*R*h/L)@res.x
    A4=A[Pa-1]
    DebA=(A4*alpha*R*R*h/L)@res.x
    Prix=Pf.T@DebC
    Couts=C.T@DebA
    print("This is res")
    print(res)
    return (-1*res.fun,DebC,DebA,Prix,Couts)#modifier le return

In [49]:
(nPoints,nConduites,X,Y,Z,A, alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi, unit)=readFile("OurReseau/Tiny.txt")
print(Analyse(nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf*np.ones(len(Dcmin)),Pi,alpha))

This is h
[1. 1. 1.]
This is x
[1. 1. 1.]
This is y
[0. 1. 0.]
This is L
[1.41421356 1.73205081 1.41421356]

This is DebMin
[[-0.         -0.57735027 -0.        ]
 [-0.         -0.         -0.70710678]]

This is DebMax
[[ 0.          0.57735027  0.        ]
 [ 0.          0.          0.70710678]
 [ 0.70710678 -0.         -0.        ]]

[[1.  0.  0. ]
 [0.  1.  0. ]
 [0.  0.  0.5]]
[ 0.35355339 -0.57735027 -0.70710678]
This is c :
[ 0.35355339 -0.57735027 -0.70710678]

This is Aub
[[ 0.          0.57735027  0.        ]
 [ 0.          0.          0.70710678]
 [ 0.70710678 -0.         -0.        ]
 [-0.         -0.57735027 -0.        ]
 [-0.         -0.         -0.70710678]]

This is b
[ 1.5  1.   3.  -1.  -1. ]

This is Aeq
[[ 0.70710678 -0.57735027 -0.70710678]]

This is beq
[0.]

This is Bounds
[[0. 1.]
 [0. 1.]
 [0. 1.]]

     fun: 1.2928932188134525
 message: 'Optimization failed. Unable to find a feasible starting point.'
     nit: 2
  status: 2
 success: False
       x: nan


ValueError: Scalar operands are not allowed, use '*' instead

In [ ]:
B = [1, 2, 3, 4, 5]
A = np.outer(np.ones(2),B)
print(B[2:3])